In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
fridge = pd.read_excel('emlak_gercek.xlsx')
df = fridge.copy()
df = df.dropna()
y = df["Fiyat"].astype('int32')
X_ = df.drop(['Fiyat'], axis=1).astype('int32')
X = pd.DataFrame(X_)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                   random_state=42)

In [3]:
knn_model = KNeighborsRegressor().fit(X_train, y_train)

In [4]:
knn_model

KNeighborsRegressor()

In [5]:
knn_model.n_neighbors

5

In [6]:
knn_model.effective_metric_

'euclidean'

In [7]:
y_pred = knn_model.predict(X_test)

In [8]:
np.sqrt(mean_squared_error(y_test, y_pred))

4733362.244702314

In [9]:
RMSE = [] 

for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train) 
    rmse = np.sqrt(mean_squared_error(y_train,y_pred)) 
    RMSE.append(rmse) 
    print("k =" , k , "için RMSE değeri: ", rmse)


k = 1 için RMSE değeri:  0.0
k = 2 için RMSE değeri:  2959669.13353785
k = 3 için RMSE değeri:  3836314.2367944876
k = 4 için RMSE değeri:  3906794.793197299
k = 5 için RMSE değeri:  3819891.62368505
k = 6 için RMSE değeri:  3964730.393533808
k = 7 için RMSE değeri:  4239090.2936713
k = 8 için RMSE değeri:  4391436.420455721
k = 9 için RMSE değeri:  4479798.595027686
k = 10 için RMSE değeri:  4538804.003061803


In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
knn_params = {'n_neighbors': np.arange(1,30,1)}

In [12]:
knn = KNeighborsRegressor()

In [13]:
knn_cv_model = GridSearchCV(knn, knn_params, cv = 10)

In [14]:
knn_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

In [15]:
knn_cv_model.best_params_["n_neighbors"]

4

In [16]:
RMSE = [] 
RMSE_CV = []
for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train) 
    rmse = np.sqrt(mean_squared_error(y_train,y_pred)) 
    rmse_cv = np.sqrt(-1*cross_val_score(knn_model, X_train, y_train, cv=10, 
                                         scoring = "neg_mean_squared_error").mean())
    RMSE.append(rmse) 
    RMSE_CV.append(rmse_cv)
    print("k =" , k , "için RMSE değeri: ", rmse, "RMSE_CV değeri: ", rmse_cv )


k = 1 için RMSE değeri:  0.0 RMSE_CV değeri:  5982187.433218376
k = 2 için RMSE değeri:  2959669.13353785 RMSE_CV değeri:  5613997.772463688
k = 3 için RMSE değeri:  3836314.2367944876 RMSE_CV değeri:  5105546.338330004
k = 4 için RMSE değeri:  3906794.793197299 RMSE_CV değeri:  4829896.922129145
k = 5 için RMSE değeri:  3819891.62368505 RMSE_CV değeri:  4884304.004380885
k = 6 için RMSE değeri:  3964730.393533808 RMSE_CV değeri:  4981847.058658641
k = 7 için RMSE değeri:  4239090.2936713 RMSE_CV değeri:  5053489.591682974
k = 8 için RMSE değeri:  4391436.420455721 RMSE_CV değeri:  5024912.807960889
k = 9 için RMSE değeri:  4479798.595027686 RMSE_CV değeri:  5046192.710099386
k = 10 için RMSE değeri:  4538804.003061803 RMSE_CV değeri:  5088355.433897832


In [17]:
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"])

In [18]:
knn_tuned.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=4)

In [19]:
np.sqrt(mean_squared_error(y_test, knn_tuned.predict(X_test)))

4908190.682791962

In [21]:
y_pred = knn_tuned.predict(X_test)

In [22]:
print("r^2 :", r2_score(y_test,y_pred))

r^2 : 0.6026914359803256
